In [1]:
!nvidia-smi


Sat Oct  4 21:49:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
from astropy.io import fits
from astropy.visualization import (ImageNormalize, MinMaxInterval, ZScaleInterval, LinearStretch)
from astropy.visualization import make_lupton_rgb
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import cupy as cp
from skimage.color import rgb2hsv
from google.colab import drive
# drive.mount('/content/drive')
os.listdir('/content/drive/My Drive/NASA IMAGES ASSETS/')

b_channel_hdul = fits.open('/content/drive/My Drive/NASA IMAGES ASSETS/h_m51_b_s05_drz_sci.fits')
h_channel_hdul = fits.open('/content/drive/My Drive/NASA IMAGES ASSETS/h_m51_h_s05_drz_sci.fits')
v_channel_hdul = fits.open('/content/drive/My Drive/NASA IMAGES ASSETS/h_m51_v_s05_drz_sci.fits')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/NASA IMAGES ASSETS/'

In [ ]:
b_channel = b_channel_hdul[0].data
h_channel = h_channel_hdul[0].data
v_channel = v_channel_hdul[0].data

In [ ]:
b_channel = np.array(b_channel)
h_channel = np.array(h_channel)
v_channel = np.array(v_channel)
b_channel.max()

In [ ]:
b_ch_normalised = ImageNormalize(b_channel, interval=ZScaleInterval(), stretch=LinearStretch())
h_ch_normalised = ImageNormalize(h_channel, interval=ZScaleInterval(), stretch=LinearStretch())
v_ch_normalised = ImageNormalize(v_channel, interval=ZScaleInterval(), stretch=LinearStretch())

b_norm_gpu = cp.array(b_ch_normalised(b_channel))
h_norm_gpu = cp.array(h_ch_normalised(h_channel))
v_norm_gpu = cp.array(v_ch_normalised(v_channel))

rgb_image = cp.stack([b_norm_gpu, h_norm_gpu, v_norm_gpu], axis=-1)
intensity_image = cp.average(rgb_image, axis=-1)
rgb_image = np.transpose(rgb_image, axes=(1, 0, 2))
# hsv_image = rgb2hsv(rgb_image)

In [ ]:
# plt.imshow(rgb_image.get())
# plt.axis('off')
# plt.show()

In [ ]:
def select_zone(np_image: np.array, x: int, y: int, zoom_level: int, w_resolution: int = 1920, h_resolution: int = 1080):
    h_img, w_img = np_image.shape[:2]
    h_ct = h_img / h_resolution
    w_ct = w_img / w_resolution

    section_height = int(h_img // zoom_level)
    section_width = int(w_img // zoom_level)

    if section_height <= 0 or section_width <= 0:
        return None

    y_end = min(y + section_height, h_img)
    x_end = min(x + section_width, w_img)

    if y_end > y and x_end > x:
        sectioned_array = np_image[y:y_end, x:x_end, :]
    else:
        return None

    return sectioned_array, w_resolution, h_resolution

In [ ]:
def compress_zone(np_image_section: np.array, w_resolution: int, h_resolution: int, quality: int = 95):
    if np_image_section is None or np_image_section.size == 0:
        return b''

    if np.issubdtype(np_image_section.dtype, np.floating):
        np_image_section = np.clip(np_image_section, 0, 1) * 255.0
        np_image_section = np_image_section.astype(np.uint8)

    if np_image_section.shape[:2] == (1, 1):
        pass

    try:
        jpg_img = Image.fromarray(np_image_section)
        jpg_img = jpg_img.resize((w_resolution, h_resolution))
        buf = BytesIO()
        jpg_img.save(buf, format="JPEG", quality=quality)
        return buf.getvalue()
    except Exception as e:
        print(f"Compression failed: {e}")
        return b''

In [ ]:
def get_zone_image(np_image: np.array, x: int, y: int, zoom_level: int, w_resolution: int = 1920, h_resolution: int = 1080):
  selected = select_zone(np_image=np_image,
                            x=x,
                            y=y,
                            zoom_level=zoom_level,
                            w_resolution=w_resolution,
                            h_resolution=h_resolution)
  if selected is None:
      return b''
  sectioned_array, wr, hr = selected
  return compress_zone(sectioned_array, wr, hr)

In [ ]:
def save_compression_levels(np_image: np.array, filename: str, exit_w_dim: int = 1920, exit_h_dim: int = 1080):
    w_img = np_image.shape[1]
    h_img = np_image.shape[0]

    max_zoom = int(max(w_img / exit_w_dim, h_img / exit_h_dim)) + 1
    for zoom_level in range(1, max_zoom+1):
        x = 0
        x_c = 0
        step_x = max(1, w_img / zoom_level)
        step_y = max(1, h_img / zoom_level)

        while x < w_img:
            y = 0
            y_c = 0
            x = int(step_x * x_c)
            while y < h_img:
                y = int(step_y * y_c)
                zone_data = get_zone_image(np_image=np_image, x=x, y=y, w_resolution=exit_w_dim, h_resolution=exit_h_dim, zoom_level=zoom_level)
                if zone_data:
                  output_filename = f'{filename}_compression_level_{zoom_level}_x_{x}_y_{y}.jpg'
                  with open(output_filename, 'wb') as f:
                      f.write(get_zone_image(np_image=np_image, x=x, y=y, w_resolution=exit_w_dim, h_resolution=exit_h_dim, zoom_level = zoom_level))
                  print(f"Saved: {output_filename}, also y_step {step_y}, also x_step {step_x}")
                y_c += 1
            x_c += 1

    # if max_zoom % 2 == 0:
        # x = 0
        # x_c = 0
        # step_x = max(1, exit_w_dim / max_zoom)
        # step_y = max(1, exit_h_dim / max_zoom)
#
        # while x + step_x <= w_img:
            # y = 0
            # y_c = 0
            # x = int(step_x * x_c)
            # while y + step_y <= h_img:
                # y = int(step_y * y_c)
                # zone_data = get_zone_image(np_image=np_image, x=x, y=y, w_resolution=exit_w_dim, h_resolution=exit_h_dim, zoom_level=max_zoom)
                # if zone_data:
                  # output_filename = f'{filename}_compression_level_{max_zoom}_x_{x}_y_{y}.jpg'
                  # with open(output_filename, 'wb') as f:
                      # f.write(zone_data)
                  # print(f"Saved: {output_filename}, also y_step {step_y}, also x_step {step_x}")
                # y_c += 1
            # x_c += 1

In [ ]:
np_rgb_image = rgb_image.get()
save_compression_levels(np_rgb_image, "test")
# save width and height to a file

In [ ]:
def get_preprocessed_image(x: int, y: int, zoom: int):
  img = Image.open(f'test_compression_level_{zoom}_x_{int(x)}_y_{int(y)}.jpg')
  print(img)
  return img

def get_stitched_image(x: int, y: int, zoom_level: float, w_resolution: int, h_resolution: int, w_img: int, h_img: int):
  w_unit = (w_img / zoom_level)
  h_unit = (h_img / zoom_level)

  x_tl = (x // w_unit) * w_unit # multiplu de w_resolution, divizor de w_img, divided by zoom_level
  y_tl = (y // h_unit) * h_unit # multiplu de w_resolution, divizor de w_img, divided by zoom_level

  x_tr = x_tl + w_unit
  y_tr = y_tl

  x_bl = x_tl
  y_bl = y_tl + h_unit

  x_br = x_tl + w_unit
  y_br = y_tl + h_unit

  img_tl = get_preprocessed_image(x_tl, y_tl, zoom_level)
  img_tr = get_preprocessed_image(x_tr, y_tr, zoom_level)
  img_bl = get_preprocessed_image(x_bl, y_bl, zoom_level)
  img_br = get_preprocessed_image(x_br, y_br, zoom_level)

  stitched_img = Image.new("RGB", (int(w_unit) * 2, int(h_unit) * 2))

  stitched_img.paste(img_tl, (0, 0))
  stitched_img.paste(img_tr, (w_resolution, 0))
  stitched_img.paste(img_bl, (0, h_resolution))
  stitched_img.paste(img_br, (w_resolution, h_resolution))

  x_offset = x-x_tl
  y_offset = y-y_tl

  cropped_img = stitched_img.crop((x_offset, y_offset, x_offset + w_unit, y_offset + h_unit))
  cropped_img = cropped_img.resize((1920, 1080))

  return cropped_img

get_stitched_image(x=400, y=50, zoom_level=3, w_resolution=1920, h_resolution=1080, w_img=np_rgb_image.shape[1], h_img=np_rgb_image.shape[0])

In [ ]:
# def save_compression_levels(np_image: np.array, filename: str, exit_w_dim: int = 1920, exit_h_dim: int = 1080):
#     if np_image.ndim < 3 or np_image.shape[2] != 3:
#         raise ValueError("Input must be a 3-channel RGB image array")

#     h_img, w_img = np_image.shape[:2]
#     max_zoom = int(max(w_img / exit_w_dim, h_img / exit_h_dim)) + 1

#     for zoom_level in range(1, max_zoom + 1):
#         step_x = max(1, exit_w_dim // zoom_level)
#         step_y = max(1, exit_h_dim // zoom_level)

#         x = 0
#         while x < w_img:
#             y = 0
#             while y < h_img:
#                 zone_data = get_zone_image(np_image=np_image, x=x, y=y, w_resolution=exit_w_dim, h_resolution=exit_h_dim, zoom_level=zoom_level)
#                 if zone_data:
#                     output_filename = f'{filename}_zoom_{zoom_level}_x_{x}_y_{y}.jpg'
#                     with open(output_filename, 'wb') as f:
#                         f.write(zone_data)
#                     print(f"Saved: {output_filename}")
#                 y += step_y
#             x += step_x